# Real / Fake Job Posting Prediction

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing the Data into pandas Data Frame


In [2]:
df=pd.read_csv('/home/ubuntu/Documents/fake_job_postings.csv.zip')

In [3]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


Remove Unrelated Attributes

In [4]:
df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [5]:
df = df.drop(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function'], axis=1)

In [6]:
df.head()

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0



DropNa

In [7]:
df.dropna(inplace=True)


Change Cols Names

In [8]:
df.columns = ['data', 'labels']
df.head()

,data,labels
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0


Lowecasing All Letters

In [9]:
df['data'] = df['data'].str.lower()
df.data[0]

'food52, a fast-growing, james beard award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its new york city headquarters.reproducing and/or repackaging existing food52 content for a number of partner sites, such as huffington post, yahoo, buzzfeed, and more in their various content management systemsresearching blogs and websites for the provisions by food52 affiliate programassisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriessupporting with pr &amp; events when neededhelping with office administrative work, such as filing, mailing, and preparing for meetingsworking with developers to document bugs and suggest improvements to the sitesupporting the marketing and executive staff'



Text Pre-processing


In [10]:


import nltk 

nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
df['data'][1]

"organised - focused - vibrant - awesome!do you have a passion for customer service? slick typing skills? maybe account management? ...and think administration is cooler than a polar bear on a jetski? then we need to hear you!\xa0we are the cloud video production service and opperating on a glodal level. yeah, it's pretty cool. serious about\xa0delivering a world class product and excellent customer service.our rapidly expanding business is looking for a talented project manager to manage the successful delivery of video projects, manage client communications and drive the production process. work with some of the coolest brands on the planet and learn from a global team that are representing nz is a huge way!we are entering the next growth stage of our business and growing quickly internationally. \xa0therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 seconds, the worlds cloud video production service -\xa0http://90

In [12]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
stops = set(stopwords.words("english"))    
def identify_tokens(row):
    review = row
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    meaningful_words = [w for w in token_words if not w in stops]
    stemmed_list = [stemming.stem(word) for word in meaningful_words]
    joined_words = (' '.join(stemmed_list))
    return joined_words

In [13]:
df['processed'] = df['data'].apply(identify_tokens)

In [14]:
df['processed'][1]

'organis focus vibrant awesom passion custom servic slick type skill mayb account manag think administr cooler polar bear jetski need hear cloud video product servic opper glodal level yeah pretti cool seriou deliv world class product excel custom rapidli expand busi look talent project manag manag success deliveri video project manag client commun drive product process work coolest brand planet learn global team repres nz huge way enter next growth stage busi grow quickli intern therefor posit burst opportun right person enter busi right time second world cloud video product servic http second world cloud video product servic enabl brand agenc get high qualiti onlin video content shot produc anywher world fast afford manag seamlessli cloud purchas publish second remov hassl cost risk speed issu work regular video product compani manag everi aspect video project beauti onlin experi grow network rate video profession countri dedic product success team countri guarante video project succ

In [15]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [16]:
Y = df['labels'].to_numpy()

TF-IDF Feature Extraction

In [18]:
tfidf = TfidfVectorizer(decode_error='ignore')
X = tfidf.fit_transform(df['processed'].values.astype('U'))
X.shape

(17879, 45566)

In [19]:
df['processed'][17876]

'payrol account focu primarili payrol function approxim count employe addit individu assist variou task within account depart rapidli grow cut edg web develop ideal candid would payrol possess experi autonomi abl work minim supervis fulfil oblig fun open work environ candid must also possess expertis applic account principl practic provid support account process payrol outsourc payrol handl account analysi administr function relat resolv respond time inquiri benefit administr includ point contact third parti complet payrol gener account analysi journal perform intern payrol review ensur accuraci maintain employe personnel annual renew flexibl spend transit reimburs assist account team account payabl'

# Train/ Test Splitting


In [20]:
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.3)

Multi-Nomial Naive Bayes

In [21]:
modelNB = MultinomialNB()
modelNB.fit(xtrain,ytrain)

MultinomialNB()

In [22]:
NBPreds = modelNB.predict(xtest)
print('Confusion Matrix')
print(confusion_matrix(ytest, NBPreds))

print('Classification Report')
print(classification_report(ytest, NBPreds))

print("Accuracy:",metrics.accuracy_score(ytest, NBPreds))
print("TNR:",metrics.precision_score(ytest, NBPreds))
print("NPV:",metrics.recall_score(ytest, NBPreds))

Confusion Matrix
[[5103    9]
 [ 252    0]]
Classification Report
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      5112
           1       0.00      0.00      0.00       252

    accuracy                           0.95      5364
   macro avg       0.48      0.50      0.49      5364
weighted avg       0.91      0.95      0.93      5364

Accuracy: 0.9513422818791947
TNR: 0.0
NPV: 0.0


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

Support Vector Machine

In [24]:
clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(xtrain,ytrain)

SVC(kernel='linear')

In [25]:
SVMpred = clf.predict(xtest)

In [26]:
print('Confusion Matrix')
print(confusion_matrix(ytest, SVMpred))
print('Classification Report')
print(classification_report(ytest, SVMpred))
print("Accuracy:",metrics.accuracy_score(ytest, SVMpred))
print("TNR:",metrics.precision_score(ytest, SVMpred))
print("NPV:",metrics.recall_score(ytest, SVMpred))

Confusion Matrix
[[5111    1]
 [ 145  107]]
Classification Report
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5112
           1       0.99      0.42      0.59       252

    accuracy                           0.97      5364
   macro avg       0.98      0.71      0.79      5364
weighted avg       0.97      0.97      0.97      5364

Accuracy: 0.9727815063385533
TNR: 0.9907407407407407
NPV: 0.4246031746031746


# Decision Tree


In [30]:
DTC = DecisionTreeClassifier()
DTC = DTC.fit(xtrain,ytrain)
DTCPred = DTC.predict(xtest)

In [28]:
print('Confusion Matrix')
print(confusion_matrix(ytest, DTCPred))
print('Classification Report')
print(classification_report(ytest, DTCPred))
print("Accuracy:",metrics.accuracy_score(ytest, DTCPred))
print("TNR:",metrics.precision_score(ytest, DTCPred))
print("NPV:",metrics.recall_score(ytest, DTCPred))

Confusion Matrix
[[5049   63]
 [ 100  152]]
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5112
           1       0.71      0.60      0.65       252

    accuracy                           0.97      5364
   macro avg       0.84      0.80      0.82      5364
weighted avg       0.97      0.97      0.97      5364

Accuracy: 0.9696122296793438
TNR: 0.7069767441860465
NPV: 0.6031746031746031


In [29]:
KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(xtrain,ytrain)
KNNPred= KNN.predict(xtest)

In [32]:
print('Confusion Matrix')
print(confusion_matrix(ytest, KNNPred))
print('Classification Report')
print(classification_report(ytest, KNNPred))
print("Accuracy:",metrics.accuracy_score(ytest, KNNPred))
print("TNR:",metrics.precision_score(ytest, KNNPred))
print("NPV:",metrics.recall_score(ytest, KNNPred))

Confusion Matrix
[[5041   71]
 [  80  172]]
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5112
           1       0.71      0.68      0.69       252

    accuracy                           0.97      5364
   macro avg       0.85      0.83      0.84      5364
weighted avg       0.97      0.97      0.97      5364

Accuracy: 0.9718493661446681
TNR: 0.7078189300411523
NPV: 0.6825396825396826


# Random Forest

In [34]:
RF=RandomForestClassifier(n_estimators=100) #default number of trees in the forest model 
RF.fit(xtrain,ytrain)
RFPred=RF.predict(xtest)
print('Confusion Matrix')
print(confusion_matrix(ytest, RFPred))
print('Classification Report')
print(classification_report(ytest, RFPred))
print("Accuracy:",metrics.accuracy_score(ytest, RFPred))
print("TNR:",metrics.precision_score(ytest, RFPred))
print("NPV:",metrics.recall_score(ytest, RFPred))

Confusion Matrix
[[5112    0]
 [ 125  127]]
Classification Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5112
           1       1.00      0.50      0.67       252

    accuracy                           0.98      5364
   macro avg       0.99      0.75      0.83      5364
weighted avg       0.98      0.98      0.97      5364

Accuracy: 0.976696495152871
TNR: 1.0
NPV: 0.503968253968254
